# HPC Allocation Mode
In contrast to the [HPC Submission Mode]() which submitts individual Python functions to HPC job schedulers, the HPC Allocation Mode takes a given allocation of the HPC job scheduler and executes Python functions with the resources available in this allocation. In this regard it is similar to the [Local Mode]() as it communicates with the individual Python processes using the [zero message queue](https://zeromq.org/), still it is more advanced as it can access the computational resources of all compute nodes of the given HPC allocation and also provides the option to assign GPUs as accelerators for parallel execution.

Available Functionality: 
* Submit Python functions with the [submit() function or the map() function]().
* Support for parallel execution, either using the [message passing interface (MPI)](), [thread based parallelism]() or by [assigning dedicated GPUs]() to selected Python functions. All these resources assignments are handled via the [resource dictionary parameter resource_dict]().
* Performance optimization features, like [block allocation](), [dependency resolution]() and [caching]().

The only parameter the user has to change is the `backend` parameter. 

## SLURM
With the [Simple Linux Utility for Resource Management (SLURM)](https://slurm.schedmd.com/) currently being the most commonly used job scheduler, executorlib provides an interface to submit Python functions to SLURM. Internally, this is based on the [srun](https://slurm.schedmd.com/srun.html) command of the SLURM scheduler, which creates job steps in a given allocation. Given that all resource requests in SLURM are communicated via a central database a large number of submitted Python functions and resulting job steps can slow down the performance of SLURM. To address this limitation it is recommended to install the hierarchical job scheduler [flux](https://flux-framework.org/) in addition to SLURM, to use flux for distributing the resources within a given allocation. This configuration is discussed in more detail below in the section [SLURM with flux]().

In [1]:
from executorlib import Executor

```python
with Executor(backend="slurm_allocation") as exe:
    future = exe.submit(sum, [1, 1])
    print(future.result())
```

## SLURM with Flux 
As discussed in the installation section it is important to select the [flux](https://flux-framework.org/) version compatible to the installation of a given HPC cluster. Which GPUs are available? Who manufactured these GPUs? Does the HPC use [mpich](https://www.mpich.org/) or [OpenMPI](https://www.open-mpi.org/) or one of their commercial counter parts like cray MPI or intel MPI? Depending on the configuration different installation options can be choosen, as explained in the [installation section](). 

Afterwards flux can be started in an [sbatch](https://slurm.schedmd.com/sbatch.html) submission script using:
```
srun flux start python <script.py>
```
In this Python script `<script.py>` the `"flux_allocation"` backend can be used.

### Resource Assignment
Independent of the selected backend [local mode](), [HPC submission mode]() or HPC allocation mode the assignment of the computational resoruces remains the same. They can either be specified in the `submit()` function by adding the resource dictionary parameter [resource_dict]() or alternatively during the initialization of the `Executor` class by adding the resource dictionary parameter [resource_dict]() there. 

This functionality of executorlib is commonly used to rewrite individual Python functions to use MPI while the rest of the Python program remains serial.

In [2]:
def calc_mpi(i):
    from mpi4py import MPI

    size = MPI.COMM_WORLD.Get_size()
    rank = MPI.COMM_WORLD.Get_rank()
    return i, size, rank

Depending on the choice of MPI version, it is recommended to specify the pmi standard which [flux](https://flux-framework.org/) should use internally for the resource assignment. For example for OpenMPI >=5 `"pmix"` is the recommended pmi standard.

In [3]:
with Executor(backend="flux_allocation", flux_executor_pmi_mode="pmix") as exe:
    fs = exe.submit(calc_mpi, 3, resource_dict={"cores": 2})
    print(fs.result())

[(3, 2, 0), (3, 2, 1)]


### Block Allocation
The block allocation for the HPC allocation mode follows the same implementation as the [block allocation for the local mode](). It starts by defining the initialization function `init_function()` which returns a dictionary which is internally used to look up input parameters for Python functions submitted to the `Executor` class. Commonly this functionality is used to store large data objects inside the Python process created for the block allocation, rather than reloading these Python objects for each submitted function.   

In [4]:
def init_function():
    return {"j": 4, "k": 3, "l": 2}

In [5]:
def calc_with_preload(i, j, k):
    return i + j + k

In [6]:
with Executor(
    backend="flux_allocation", flux_executor_pmi_mode="pmix", max_workers=2, init_function=init_function, block_allocation=True
) as exe:
    fs = exe.submit(calc_with_preload, 2, j=5)
    print(fs.result())


10


In this example the parameter `k` is used from the dataset created by the initialization function while the parameters `i` and `j` are specified by the call of the `submit()` function. 

When using the block allocation mode, it is recommended to set either the maxium number of workers using the `max_workers` parameter or the maximum number of CPU cores using the `max_cores` parameter to prevent oversubscribing the available resources. 

### Dependencies
Python functions with rather different computational resource requirements should not be merged into a single function. So to able to execute a series of Python functions which each depend on the output of the previous Python function executorlib internally handles the dependencies based on the [concurrent futures future](https://docs.python.org/3/library/concurrent.futures.html#future-objects) objects from the Python standard library. This implementation is independent of the selected backend and works for HPC allocation mode just like explained in the [local mode section]().  

In [7]:
def add_funct(a, b):
    return a + b

In [8]:
with Executor(backend="flux_allocation", flux_executor_pmi_mode="pmix") as exe:
    future = 0
    for i in range(1, 4):
        future = exe.submit(add_funct, i, future)
    print(future.result())

6


### Caching
Finally, also the caching is available for HPC allocation mode, in analogy to the [local mode](). Again this functionality is not designed to identify function calls with the same parameters, but rather provides the option to reload previously cached results even after the Python processes which contained the executorlib `Executor` class is closed. As the cache is stored on the file system, this option can decrease the performance of executorlib. Consequently the caching option should primarily be used during the prototyping phase. 

In [9]:
with Executor(backend="flux_allocation", flux_executor_pmi_mode="pmix", cache_directory="./cache") as exe:
    future_lst = [exe.submit(sum, [i, i]) for i in range(1, 4)]
    print([f.result() for f in future_lst])

[2, 4, 6]


In [10]:
import os
import shutil

cache_dir = "./cache"
if os.path.exists(cache_dir):
    print(os.listdir(cache_dir))
    try:
        shutil.rmtree(cache_dir)
    except OSError:
        pass

['sumd1bf4ee658f1ac42924a2e4690e797f4.h5out', 'sum5171356dfe527405c606081cfbd2dffe.h5out', 'sumb6a5053f96b7031239c2e8d0e7563ce4.h5out']


### Nested executors
The hierarchical nature of the [flux](https://flux-framework.org/) job scheduler allows the creation of additional executorlib Executors inside the functions submitted to the Executor. This hierarchy can be beneficial to separate the logic to saturate the available computational resources. 

In [11]:
def calc_nested():
    from executorlib import Executor
    
    with Executor(backend="flux_allocation", flux_executor_pmi_mode="pmix") as exe:
        fs = exe.submit(sum, [1, 1])
        return fs.result()

In [12]:
with Executor(backend="flux_allocation", flux_executor_pmi_mode="pmix") as exe:
    fs = exe.submit(calc_nested)
    print(fs.result())

2


### Resource Monitoring
For debugging it is commonly helpful to keep track of the computational resources. [flux](https://flux-framework.org/) provides a number of features to analyse the resource utilization, so here only the two most commonly used ones are introduced. Starting with the option to list all the resources available in a given allocation with the `flux resource list` command:

In [13]:
! flux resource list

     STATE NNODES   NCORES    NGPUS NODELIST
      free      1        2        0 fedora
 allocated      0        0        0 
      down      0        0        0 


Followed by the list of jobs which were executed in a given flux session. This can be retrieved using the `flux jobs -a` command:

In [14]:
! flux jobs -a

       JOBID USER     NAME       ST NTASKS NNODES     TIME INFO
    ƒDqBpVYK jan      python     CD      1      1   0.695s fedora
    ƒDxdEtYf jan      python     CD      1      1   0.225s fedora
    ƒDVahzPq jan      python     CD      1      1   0.254s fedora
    ƒDSsZJXH jan      python     CD      1      1   0.316s fedora
    ƒDSu3Hod jan      python     CD      1      1   0.277s fedora
    ƒDFbkmFD jan      python     CD      1      1   0.247s fedora
    ƒD9eKeas jan      python     CD      1      1   0.227s fedora
    ƒD3iNXCs jan      python     CD      1      1   0.224s fedora
    ƒCoZ3P5q jan      python     CD      1      1   0.261s fedora
    ƒCoXZPoV jan      python     CD      1      1   0.261s fedora
    ƒCZ1URjd jan      python     CD      2      1   0.360s fedora


## Flux
While the number of HPC clusters which use [flux](https://flux-framework.org/) as primary job scheduler is currently still limited the setup and functionality provided by executorlib for running [SLURM with flux]() also applies to HPCs which use [flux](https://flux-framework.org/) as primary job scheduler.